## Webcam: Eye aspect ratio + CNN

In [1]:

from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import tensorflow as tf
import numpy as np
import time
from playsound import playsound

In [2]:

# load face detector and 68_face_landmarks feature extraction model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# eye coordinate
LEFT_EYE = list(range(42, 48))
RIGHT_EYE = list(range(36, 42))


In [3]:

# pretrained CNN
model = tf.keras.models.load_model("fatigue_detection_CNN.h5")

In [4]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [6]:
# parameters
EYE_AR_THRESH = 0.2  # EAR threshold
EYE_AR_CONSEC_FRAMES = 20  # activate warning after closing eyes for more than 20 frames
IMG_SIZE = 145  # CNN input size

frame_counter = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # dlib requires gray pic
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # faces detector
    faces = detector(gray)
    
    for face in faces:
        shape = predictor(gray, face)  # feature points
        shape_np = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])

        left_eye = shape_np[LEFT_EYE]
        right_eye = shape_np[RIGHT_EYE]

        # EAR
        left_EAR = eye_aspect_ratio(left_eye)
        right_EAR = eye_aspect_ratio(right_eye)
        avg_EAR = (left_EAR + right_EAR) / 2.0

        # plot eyes zone
        cv2.polylines(frame, [left_eye], True, (0, 255, 0), 1)
        cv2.polylines(frame, [right_eye], True, (0, 255, 0), 1)

        # CNN preprocessing
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())  # get face region
        face_img = frame[y:y+h, x:x+w]  # cut
        face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE)) / 255.0 
        face_img = np.expand_dims(face_img, axis=0)
        
        cnn_prediction = model.predict(face_img)[0][0]
        
        # if cnn predicts as fatigue or ear less than threshold
        if avg_EAR < EYE_AR_THRESH:
            frame_counter += 1

            if frame_counter >= EYE_AR_CONSEC_FRAMES  or cnn_prediction < 0.5:
                status = "Fatigue Detected!"
                color = (0, 0, 255)
                # playsound("alarm.wav")
            else:
                status = "Active"
                color = (0, 255, 0)
        else:
            frame_counter = 0
            status = "Active"
            color = (0, 255, 0)
        
        
        # show informations on frame
        cv2.putText(frame, status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # results
    cv2.imshow("Fatigue Detection", frame)
    
    # press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━